In [1]:
#base imports
import os
import sys

import numpy as np
import pandas as pd

from IPython.core.display import HTML

import torch
import torch.nn.functional as F
from torch_sparse import coalesce

In [6]:
#RDKit
import rdkit
from rdkit import Chem
from rdkit.Chem.rdchem import HybridizationType
from rdkit.Chem.rdchem import BondType as bt
from rdkit.Chem import PandasTools
from rdkit import RDLogger

RDLogger.logger().setLevel(RDLogger.CRITICAL)

In [4]:
#fcts
def show(df):
    """Render the molecules within a DataFrame correctly"""
    return HTML(df.to_html(notebook=True))


In [3]:
#check GPU
print(torch.cuda.is_available())

True


In [5]:
assay = 1511

In [7]:
#setup dataset
sdfpath = os.path.join(os.getcwd(), f'{assay}_compounds.sdf')
backup_df = PandasTools.LoadSDF(sdfpath)
dataframe = backup_df.copy()

[10:41:00] WARNING: not removing hydrogen atom without neighbors
[10:41:01] WARNING: not removing hydrogen atom without neighbors


In [8]:
show(dataframe.head(5))

NameError: name 'HTML' is not defined